## import libs

In [1]:
import pandas as pd
import json
import numpy as np
import folium
from folium.plugins import MarkerCluster
import geopandas as gpd
from shapely.geometry import Point
import random
import pyproj
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")


In [2]:
# Define the prefix you want to search for
prefix = 'R'

# Get a list of all variables in the current Jupyter session
all_variables = %who_ls

# Filter variables that start with the specified prefix
filtered_variables = [var for var in all_variables if var.startswith(prefix)]

# Print the filtered variables and their values
for var in filtered_variables:
    print(f"{var}: {globals()[var]}")


In [3]:
#!pip install geopandas shapely pyproj folium

## add some user-defined Functions

In [4]:
def drop_zeros(df, columns=None):
    if columns is None:
        columns = df.columns
    return df[~(df[columns] == 0).any(axis=1)]

shape = 'city_bound_UTM/Eslamshar_elec_mark.shp'
#elec_auth = 'path/to/there'

## I - data pre-filtering by previous phase and split it into two parts

### load access raw dataframe 

In [5]:
access_column_name = ['bill_ID','agent_x', 'agent_y', 'counter_x', 'counter_y']
access_df = pd.read_csv('access_table.txt', delimiter = ',', names=access_column_name)
access_df.head()

,bill_ID,agent_x,agent_y,counter_x,counter_y
0,8256564304224,520572.0,3934172.0,NaN,NaN
1,8442572304223,520178.0,3931545.0,NaN,NaN
2,8685393504228,520390.0,3932362.0,520349.0,3932309.0
3,8520916604222,520409.0,3932472.0,NaN,NaN
4,8521526604228,520411.0,3932471.0,NaN,NaN


In [6]:
access_df.tail()

,bill_ID,agent_x,agent_y,counter_x,counter_y
4273985,8632699404220,521586.0,3931862.0,NaN,NaN
4273986,8632699404220,521588.0,3931862.0,NaN,NaN
4273987,8632699404220,521587.0,3931863.0,NaN,NaN
4273988,8632699404220,521588.0,3931867.0,NaN,NaN
4273989,8632699404220,521591.0,3931864.0,NaN,NaN


### extract table dimention for further reporting 

In [7]:
R_access_rows = (access_df.shape[0]) -1 
R_access_ID_unique = len(access_df['bill_ID'].unique())
# Display the number of rows
print("Number of records:", R_access_rows)
print("Number of unique IDs:", R_access_ID_unique)

Number of records: 4273989
Number of unique IDs: 189875


### load the DBSCAN block_accu for filtering prevous data

In [8]:
dbscan_df = pd.read_csv('dbscan.csv', delimiter=',')

In [9]:
dbscan_df.head()

,BILL_IDENT,NUM
0,8717022804228,15
1,8259334804227,15
2,8494666504226,15
3,8385632004220,15
4,8492681804225,15


In [10]:
dbscan_df.tail()

,BILL_IDENT,NUM
134578,8699301304229,16
134579,8256793904227,15
134580,8255283504228,15
134581,8746657504220,7
134582,8499709304228,13


### extract table dimention for further reporting 

In [11]:
R_dbscan_rows_uniqueID = (dbscan_df.shape[0]) - 1
print("Number of records and Unique IDs:", R_dbscan_rows_uniqueID)

Number of records and Unique IDs: 134582


### colloct unique bill_IDs form DBSCAN data ( the IDs are unique but we do it to make sure)

In [12]:
access_ID_unique = access_df['bill_ID'].unique()
bill_ident_value = dbscan_df['BILL_IDENT'].unique()

In [13]:
print(bill_ident_value)

[8717022804228 8259334804227 8494666504226 ... 8255283504228 8746657504220
 8499709304228]


### remove common IDs from data table 

In [14]:
flt_by_dbscan_df =access_df[~access_df['bill_ID'].isin(bill_ident_value)]
#access_bill_id_filterd_value = filter_access_df['bill_ID'].unique()
flt_by_dbscan_df.reset_index(drop=True, inplace=True)

In [15]:
flt_by_dbscan_df.tail()

,bill_ID,agent_x,agent_y,counter_x,counter_y
1111862,8613350104225,526675.0,3933960.0,NaN,NaN
1111863,8613350104225,527572.0,3931762.0,NaN,NaN
1111864,8613350104225,526662.0,3934166.0,NaN,NaN
1111865,8613350104225,526663.0,3933935.0,NaN,NaN
1111866,8613350104225,526667.0,3933964.0,NaN,NaN


### extract table dimention for further reporting 

In [16]:
R_flt_by_dbscan_rows = flt_by_dbscan_df.shape[0]
R_flt_by_dbscan_ID_unique = len(flt_by_dbscan_df['bill_ID'].unique())
print("Number of records:", R_flt_by_dbscan_rows)
print("Number of unique IDs:", R_flt_by_dbscan_ID_unique)

Number of records: 1111867
Number of unique IDs: 55640


In [17]:
flt_by_dbscan_df.to_csv('filtered_by_dbscan.csv', index=False)

### replace zeros in agent with nan

In [18]:
flt_by_dbscan_df.replace(0, np.nan, inplace=True)
flt_by_dbscan_df.to_csv('filtered_by_dbscan_zer2null.csv', index=False)


###  split data into two part ( agent base or counter base ) extract IDs with counter X,Y ( they may have agent X,Y as well)
### create two CSVs file that are included or excluded rows with counter coordinates 

In [19]:
# Extract rows with non-null values for counter_x and counter_y
counter_data_df = flt_by_dbscan_df.dropna(subset=['counter_x', 'counter_y'])
counter_data_df.to_csv('counter_data.csv', index=False)

In [20]:
R_counter_data_rows = counter_data_df.shape[0]
R_counter_data_ID_unique = len(counter_data_df['bill_ID'].unique())
print("Number of records:", R_counter_data_rows)
print("Number of unique IDs:", R_counter_data_ID_unique)

Number of records: 252781
Number of unique IDs: 17150


In [21]:
# Select rows where counter_x and counter_y are NaN
agent_data_df = flt_by_dbscan_df[pd.isnull(flt_by_dbscan_df['counter_x']) & pd.isnull(flt_by_dbscan_df['counter_y'])]

# Save the  DataFrame if needed
agent_data_df.to_csv('agent_data.csv', index=False)

In [22]:
R_agent_data_df_rows = agent_data_df.shape[0]
R_agent_data_ID_unique = len(agent_data_df['bill_ID'].unique())
print("Number of records:", R_agent_data_df_rows)
print("Number of unique IDs:", R_agent_data_ID_unique)

Number of records: 857326
Number of unique IDs: 38399


## II - Process table contains counter coordinates 

### display counters 

In [23]:
# Read the filtered CSV data
data = pd.read_csv('counter_data.csv')
df = data.drop_duplicates(subset='bill_ID')
# Sample a subset of the data
sampled_data = df#.sample(n=16000)  # Adjust the number of samples as needed

# Convert UTM coordinates to latitude and longitude
utm_proj = pyproj.Proj(proj='utm', zone=39, ellps='WGS84')
lonlat_proj = pyproj.Proj(proj='latlong', datum='WGS84')
sampled_data['lon'], sampled_data['lat'] = pyproj.transform(utm_proj, lonlat_proj, sampled_data['counter_x'].values, sampled_data['counter_y'].values)

# Read the boundary shapefile
boundary_shapefile = gpd.read_file(shape) ###change the name and path as needed

# Create a Folium map centered at the mean of all points
m = folium.Map(location=[sampled_data['lat'].mean(), sampled_data['lon'].mean()],zoom_start=3.5) #,zoom_start=10)

# Plot the boundary shapefile
folium.GeoJson(boundary_shapefile).add_to(m)

# Create a MarkerCluster layer for the sampled data points
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each point in the sampled data
for idx, row in sampled_data.iterrows():
    folium.Marker(location=[row['lat'], row['lon']]).add_to(marker_cluster)

# Display the map



In [24]:
# Display the map
#m

### fix counter errors, since the oprator mistakenly entered the coordinates inverted

In [25]:
import pandas as pd

# Load the data into a DataFrame
df = pd.read_csv('counter_data.csv')

# Define the mask to identify rows where counter_x and counter_y might be reversed
# Typically, UTM x-coordinates are between 100,000 and 900,000 (for zone-specific cases)
# y-coordinates range between 0 and 10,000,000
mask = (df['counter_x'] > 900000) & (df['counter_y'] < 1000000)

# Count the number of rows with inverted coordinates
counters_inverted = mask.sum()

# Swap the counter_x and counter_y values where the mask is True
df.loc[mask, ['counter_x', 'counter_y']] = df.loc[mask, ['counter_y', 'counter_x']].values

# Display the count of corrected rows
print(f'Number of rows with inverted coordinates: {counters_inverted}')

# Optionally, save the corrected DataFrame back to a CSV file
df.to_csv('corrected_counter_data.csv', index=False)
# save the only rows that has been correceted
corrected_rows = df[mask]
corrected_rows.to_csv('Jst_fixed_counter_data.csv', index=False)


Number of rows with inverted coordinates: 86315


### extract table dimention for further reporting 

In [26]:
R_fixed_invert_counter_records = corrected_rows.shape[0]
R_fixed_invert_counter_ID_unique = len(corrected_rows['bill_ID'].unique())
print("Number of records:", R_fixed_invert_counter_records)
print("Number of unique IDs:", R_fixed_invert_counter_ID_unique)

Number of records: 86315
Number of unique IDs: 9994


### display corrected counters 

In [27]:
# Read the filtered CSV data
data = pd.read_csv('corrected_counter_data.csv')
df = data.drop_duplicates(subset='bill_ID')
# Sample a subset of the data
sampled_data = df#.sample(n=16000)  # Adjust the number of samples as needed

# Convert UTM coordinates to latitude and longitude
utm_proj = pyproj.Proj(proj='utm', zone=39, ellps='WGS84')
lonlat_proj = pyproj.Proj(proj='latlong', datum='WGS84')
sampled_data['lon'], sampled_data['lat'] = pyproj.transform(utm_proj, lonlat_proj, sampled_data['counter_x'].values, sampled_data['counter_y'].values)

# Read the boundary shapefile
boundary_shapefile = gpd.read_file(shape) ###change the name and path as needed

# Create a Folium map centered at the mean of all points
m = folium.Map(location=[sampled_data['lat'].mean(), sampled_data['lon'].mean()],zoom_start=3.5) #,zoom_start=10)

# Plot the boundary shapefile
folium.GeoJson(boundary_shapefile).add_to(m)

# Create a MarkerCluster layer for the sampled data points
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each point in the sampled data
for idx, row in sampled_data.iterrows():
    folium.Marker(location=[row['lat'], row['lon']]).add_to(marker_cluster)




In [28]:
# Display the map
#m

### remove counter coordiantes from each row since they are outsider ! 

In [29]:
df_corr = pd.read_csv('corrected_counter_data.csv')

# Displaying the first few rows of the DataFrame to ensure it loaded correctly
print(df_corr)

              bill_ID   agent_x    agent_y  counter_x  counter_y
0       8693290904220  521722.0  3936717.0   521717.0  3936724.0
1       8738020704223  521457.0  3936857.0   521474.0  3936848.0
2       8691350404220  520574.0  3934585.0   520632.0  3934760.0
3       8659094804222  520574.0  3934585.0   520589.0  3934621.0
4       8697350304223  520574.0  3934585.0    52092.0  3934531.0
...               ...       ...        ...        ...        ...
252776  8685114004229  520756.0  3933283.0   520761.0  3933300.0
252777  8685114004229  520749.0  3933295.0   520761.0  3933300.0
252778  8712756904220  519622.0  3937088.0   519635.0  3937080.0
252779  8712756904220  519652.0  3937079.0   519635.0  3937080.0
252780  8712756904220  519460.0  3937206.0   519635.0  3937080.0

[252781 rows x 5 columns]


In [30]:

# Load the shapefile
shapefile = gpd.read_file(shape)

# Define a function to check if a point is within the shapefile boundary
def is_within_boundary(point):
    point_geom = Point(point['counter_x'], point['counter_y'])
    return shapefile.geometry.contains(point_geom).any()

# Filter the DataFrame based on the condition and split into two DataFrames
inside_boundary = df[df.apply(is_within_boundary, axis=1)]
outside_boundary = df[~df.apply(is_within_boundary, axis=1)]

# Reset the index of the DataFrames
inside_boundary.reset_index(drop=True, inplace=True)
outside_boundary.reset_index(drop=True, inplace=True)

# Save the filtered DataFrames to separate files
inside_boundary.to_csv('counters_inside_boundary.csv', index=False)
outside_boundary.to_csv('counters_outside_boundary.csv', index=False)

# Now, inside_boundary contains rows where the counter_x and counter_y values are within the shapefile boundary, 
# and outside_boundary contains rows where they are outside the boundary.


### Inside the city report : 

In [31]:
inside_boundary_records = inside_boundary.shape[0]
inside_boundary_ID_unique = len(inside_boundary['bill_ID'].unique())
print("Number of records:", inside_boundary_records)
print("Number of unique IDs:", inside_boundary_ID_unique)

Number of records: 16581
Number of unique IDs: 16581


### outside the city report : 

In [32]:
outside_boundary_records = outside_boundary.shape[0]
outside_boundary_ID_unique = len(outside_boundary['bill_ID'].unique())
print("Number of records:", outside_boundary_records)
print("Number of unique IDs:", outside_boundary_ID_unique)

Number of records: 569
Number of unique IDs: 569


### outside the city on map :

In [33]:
# Read the filtered CSV data
data = pd.read_csv('outside_boundary.csv')
df = data.drop_duplicates(subset='bill_ID')
# Sample a subset of the data
sampled_data = df#.sample(n=16000)  # Adjust the number of samples as needed

# Convert UTM coordinates to latitude and longitude
utm_proj = pyproj.Proj(proj='utm', zone=39, ellps='WGS84')
lonlat_proj = pyproj.Proj(proj='latlong', datum='WGS84')
sampled_data['lon'], sampled_data['lat'] = pyproj.transform(utm_proj, lonlat_proj, sampled_data['counter_x'].values, sampled_data['counter_y'].values)

# Read the boundary shapefile
boundary_shapefile = gpd.read_file(shape) ###change the name and path as needed

# Create a Folium map centered at the mean of all points
m = folium.Map(location=[sampled_data['lat'].mean(), sampled_data['lon'].mean()],zoom_start=3.5) #,zoom_start=10)

# Plot the boundary shapefile
folium.GeoJson(boundary_shapefile).add_to(m)

# Create a MarkerCluster layer for the sampled data points
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each point in the sampled data
for idx, row in sampled_data.iterrows():
    folium.Marker(location=[row['lat'], row['lon']]).add_to(marker_cluster)




In [34]:
# Display the map
#m

### inside the city on map :

In [35]:
# Read the filtered CSV data
data = pd.read_csv('inside_boundary.csv')
df = data.drop_duplicates(subset='bill_ID')
# Sample a subset of the data
sampled_data = df#.sample(n=16000)  # Adjust the number of samples as needed

# Convert UTM coordinates to latitude and longitude
utm_proj = pyproj.Proj(proj='utm', zone=39, ellps='WGS84')
lonlat_proj = pyproj.Proj(proj='latlong', datum='WGS84')
sampled_data['lon'], sampled_data['lat'] = pyproj.transform(utm_proj, lonlat_proj, sampled_data['counter_x'].values, sampled_data['counter_y'].values)

# Read the boundary shapefile
boundary_shapefile = gpd.read_file(shape) ###change the name and path as needed

# Create a Folium map centered at the mean of all points
m = folium.Map(location=[sampled_data['lat'].mean(), sampled_data['lon'].mean()],zoom_start=3.5) #,zoom_start=10)

# Plot the boundary shapefile
folium.GeoJson(boundary_shapefile).add_to(m)

# Create a MarkerCluster layer for the sampled data points
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each point in the sampled data
for idx, row in sampled_data.iterrows():
    folium.Marker(location=[row['lat'], row['lon']]).add_to(marker_cluster)



In [48]:
# Display the map
#m


## III - Process table contains only agents without counter coordinates

### load table 

In [37]:
agent_table = pd.read_csv('agent_data.csv')


In [38]:
R_agent_records = agent_table.shape[0]
R_agent_records_ID_unique = len(agent_table['bill_ID'].unique())
print("Number of records:", R_agent_records)
print("Number of unique IDs:", R_agent_records_ID_unique)

Number of records: 857326
Number of unique IDs: 38399


### ploting data that is Null values and common IDs are removed and area is defined as shapefile :

In [39]:

# Sample a subset of the data
sampled_data = agent_table.sample(n=25000)  # Adjust the number of samples as needed

# Convert UTM coordinates to latitude and longitude
utm_proj = pyproj.Proj(proj='utm', zone=39, ellps='WGS84')
lonlat_proj = pyproj.Proj(proj='latlong', datum='WGS84')
sampled_data['lon'], sampled_data['lat'] = pyproj.transform(utm_proj, lonlat_proj, sampled_data['agent_x'].values, sampled_data['agent_y'].values)

# Read the boundary shapefile
boundary_shapefile = gpd.read_file(shape)
# Create a Folium map centered at the mean of all points
m = folium.Map(location=[sampled_data['lat'].mean(), sampled_data['lon'].mean()], zoom_start=10)

# Plot the boundary shapefile
folium.GeoJson(boundary_shapefile).add_to(m)

# Create a MarkerCluster layer for the sampled data points
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each point in the sampled data
for idx, row in sampled_data.iterrows():
    folium.Marker(location=[row['lat'], row['lon']]).add_to(marker_cluster)



In [49]:
# Display the map
#m


### removing outlayer data by a given shapefile and save it, then plot the data

In [41]:
# Step 1: Read the CSV file into a pandas DataFrame
csv_data = pd.read_csv('agent_data.csv')

# Step 2: Convert the DataFrame into a GeoDataFrame
geometry = [Point(xy) for xy in zip(csv_data['agent_x'], csv_data['agent_y'])]
csv_geo_df = gpd.GeoDataFrame(csv_data, crs="EPSG:32639", geometry=geometry)

# Step 3: Read the shapefile into a GeoDataFrame
shapefile = gpd.read_file(shape)
# Step 4: Perform a spatial join
joined_data = gpd.sjoin(csv_geo_df, shapefile, how="inner", op="within")

# Step 5: Filter the CSV GeoDataFrame based on the spatial join result
agent_data_crpd = csv_geo_df[csv_geo_df.index.isin(joined_data.index)]

# Step 6: Save the filtered DataFrame back to a CSV file
agent_data_crpd.to_csv('agent_data_crpd.csv', index=False)


In [42]:
R_agent_inside_boundary_records = agent_data_crpd.shape[0]
R_agent_inside_boundary_ID_unique = len(agent_data_crpd['bill_ID'].unique())
print("Number of records:", R_agent_inside_boundary_records)
print("Number of unique IDs:", R_agent_inside_boundary_ID_unique)

Number of records: 850674
Number of unique IDs: 38345


### plot a small sample of croped agent data

In [43]:

# Read the filtered CSV data
data = pd.read_csv('agent_data_crpd.csv')

# Sample a subset of the data
sampled_data = data.sample(n=50000)  # Adjust the number of samples as needed

# Convert UTM coordinates to latitude and longitude
utm_proj = pyproj.Proj(proj='utm', zone=39, ellps='WGS84')
lonlat_proj = pyproj.Proj(proj='latlong', datum='WGS84')
sampled_data['lon'], sampled_data['lat'] = pyproj.transform(utm_proj, lonlat_proj, sampled_data['agent_x'].values, sampled_data['agent_y'].values)

# Read the boundary shapefile
boundary_shapefile = gpd.read_file(shape)

# Create a Folium map centered at the mean of all points
m = folium.Map(location=[sampled_data['lat'].mean(), sampled_data['lon'].mean()], zoom_start=10)

# Plot the boundary shapefile
folium.GeoJson(boundary_shapefile).add_to(m)

# Create a MarkerCluster layer for the sampled data points
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each point in the sampled data
for idx, row in sampled_data.iterrows():
    folium.Marker(location=[row['lat'], row['lon']]).add_to(marker_cluster)




In [44]:
# Display the map
#m

In [45]:
import pandas as pd
import json

# Assuming 'filtered' is your DataFrame
data = pd.read_csv('agent_data_crpd.csv')
# Grouping by bill_ID and aggregating agent_x and agent_y into lists of tuples
grouped_data = data.groupby('bill_ID').apply(lambda x: x[['agent_x', 'agent_y']].values.tolist()).to_dict()

# Converting the dictionary to a JSON string
json_result = json.dumps(grouped_data, indent=4)

# Display the JSON result
#print(json_result)

# Optionally, save the JSON string to a file
with open('result.json', 'w') as file:
    file.write(json_result)


In [46]:
# Grouping by bill_ID and aggregating agent_x and agent_y into lists
data = pd.read_csv('agent_data_crpd.csv')
grouped = data.groupby('bill_ID').agg({
    'agent_x': list,
    'agent_y': list
})

# Calculating the standard deviation for agent_x and agent_y for each group
grouped['agent_x_std'] = data.groupby('bill_ID')['agent_x'].std()
grouped['agent_y_std'] = data.groupby('bill_ID')['agent_y'].std()

# Calculating the combined standard deviation
grouped['MD_agent_std'] = np.sqrt(grouped['agent_x_std']**2 + grouped['agent_y_std']**2)

# Calculating the average agent_x and agent_y values for each group
grouped['agent_x_avg'] = data.groupby('bill_ID')['agent_x'].mean()
grouped['agent_y_avg'] = data.groupby('bill_ID')['agent_y'].mean()

# Creating a dictionary from the grouped data
grouped_data = grouped.apply(lambda row: {
    'agent_x': row['agent_x'],
    'agent_y': row['agent_y'],
    'agent_x_std': row['agent_x_std'],
    'agent_y_std': row['agent_y_std'],
    'MD_agent_std': row['MD_agent_std'],
    'agent_x_avg': row['agent_x_avg'],
    'agent_y_avg': row['agent_y_avg']
}, axis=1).to_dict()

# Convert the dictionary to a JSON string
json_result = json.dumps(grouped_data, indent=4)

# Display the JSON result
#print(json_result)

# Optionally, save the JSON string to a file
with open('tst.json', 'w') as file:
    file.write(json_result)

In [47]:
import json

# Load the JSON data from a file
with open('tst.json', 'r') as f:
    data = json.load(f)

# Initialize the reorganized data dictionary
reorganized_data = {}

# Reorganize the data
for bill_id, info in data.items():
    agent_x = info['agent_x']
    agent_y = info['agent_y']
    
    # Combine agent_x and agent_y into coordinate pairs
    agent_coordinates = list(zip(agent_x, agent_y))
    
    # Store the reorganized data
    reorganized_data[bill_id] = {
        "agent_coordinates": agent_coordinates,
        "agent_x_std": info["agent_x_std"],
        "agent_y_std": info["agent_y_std"],
        "MD_agent_std": info["MD_agent_std"],
        "agent_x_avg": info["agent_x_avg"],
        "agent_y_avg": info["agent_y_avg"]
    }

# Convert the reorganized data dictionary to JSON
json_result = json.dumps(reorganized_data, indent=4)

# Display the JSON result
with open('reorg.json', 'w') as file:
    file.write(json_result)